# Geometric Tolerance Analysis

This notebook demonstrates the Geometric Tolerance Analysis (GTA) methodology on the centering pin mechanism introduced by [B. Wang, X. Huang, M. Chang](https://doi.org/10.1177/00368504211013227). For the theoretical background to GTA see the (upcoming) CIRP CAT 2026 conference proceeding "Geometric Tolerance Analysis: A coordinate-free framework for Functional Requirement evaluation between arbitrary feature types".

The goal of GTA is to analyse the effect of tolerances on a functional requirement (FR). In the case of the centering pin mechanism shown below, the FR is the height of the tip above the base plate.

<img src="https://www.researchgate.net/publication/351253177/figure/fig2/AS:11431281146993092@1681577954565/A-three-parts-centering-pin-mechanism_W640.jpg"/>

We therefore need a way to add up the effects of all the tolerance zones. GTA is a novel methodology based on PGA that does exactly that.

In [1]:
%pip install -q kingdon anywidget==0.9.9 ipywidgets==8.1.3

Note: you may need to restart the kernel to use updated packages.


We first create the 3D PGA algebra $\mathbb{R}_{3,0,1}$ with kingdon.

In [2]:
from kingdon import Algebra

alg = Algebra(3, 0, 1)

Then we create the symbols $\alpha_i, \beta_i, \gamma_i, u_i, v_i$ and $w_i$ to represent the various tolerance zones, using the same indices as in the paper:

In [3]:
from sympy import symbols

a1, a2, a3, a4 = symbols('a1:5')
b1 = symbols('b1')
c2, c3, c4 = symbols('c2:5')
u2, u3, u4 = symbols('u2:5')
w1, w2, w3, w4 = symbols('w1:5')

Now we can construct the bivectors representing the tolerance zones, symbolically given by
$$
\begin{aligned}
    T_{1} &= \alpha_1 \mathbf{e}_{23} + \beta_1 \mathbf{e}_{31} + w_1 \mathbf{e}_{03} \\
    %= \tfrac{t_2}{100} \mathbf{e}_{23} + \tfrac{t_2}{80} \mathbf{e}_{31} + t_1 \mathbf{e}_{03} \\
    T_{3} &= \alpha_2 \mathbf{e}_{23} + \gamma_2 \mathbf{e}_{12} + u_2 \mathbf{e}_{01} + w_2 \mathbf{e}_{03} \\
    T_{4} &= \alpha_3 \mathbf{e}_{23} + \gamma_3 \mathbf{e}_{12} + u_3 \mathbf{e}_{01} + w_3 \mathbf{e}_{03} \\
    T_{5} &= \alpha_4 \mathbf{e}_{23} + \gamma_4 \mathbf{e}_{12} + u_4 \mathbf{e}_{01} + w_4 \mathbf{e}_{03} \ .
\end{aligned}
$$

In [4]:
T1 = alg.bivector(e23=a1, e31=b1,                 e03=w1)
T3 = alg.bivector(e23=a2,         e12=c2, e01=u2, e03=w2)
T4 = alg.bivector(e23=a3,         e12=c3, e01=u3, e03=w3)
T5 = alg.bivector(e23=a4,         e12=c4, e01=u4, e03=w4)

Note that all of these bivectors are defined in their own local reference frame, so they need to be placed in the global reference frame. Like Wang et al., we use the reference frame of the tip of the pin as the global reference frame, and construct motors $M_i$ that place each tolerance zone $T_i$ in the right position in space:

In [5]:
M1 = alg.evenmv(e=1, e02=110/2, e03=55/2)
M3 = M4 = alg.evenmv(e=1, e02=85/2)
M5 = alg.evenmv(e=1)

We can now create the total contribution of the tolerance zones to the functional requirement by placing each $T_i$ at the correct location with the corresponding $M_i$ by using conjugation: $M_i T_i \widetilde{M_i}$.
In kingdon, conjugation is performed with the `>>` operator.

In [6]:
T = (M1 >> T1) + (M3 >> T3) + (M4 >> T4) + (M5 >> T5)
T

(55.0*b1 - 85.0*c2 - 85.0*c3 + u2 + u3 + u4) 𝐞₀₁ + (-55.0*a1) 𝐞₀₂ + (110.0*a1 + 85.0*a2 + 85.0*a3 + w1 + w2 + w3 + w4) 𝐞₀₃ + (c2 + c3 + c4) 𝐞₁₂ + (-b1) 𝐞₁₃ + (a1 + a2 + a3 + a4) 𝐞₂₃

We can now study the effect of these tolerances on the functional requirement, defined by the plane of the base plate
$$ f_0 = 40 \mathbf{e}_0 + \mathbf{e}_3 $$
and the point at the tip of the pin
$$ f_5 = \mathbf{e}_0^* = \mathbf{e}_{123} \ . $$

The functional requirement $F$ is given by
$$ F = f_5 \widetilde{f_0} \ . $$

In [7]:
f5 = alg.vector(e0=1).dual()
f0 = alg.vector(e0=40, e3=1)
F = f5 * ~f0
F

1 𝐞₁₂ + -40 𝐞₀₁₂₃

Hence we can calculate that the distance between the tip and the base plate is
$$ d = \lVert \langle F \rangle_4 / \langle F \rangle_2 \rVert_\infty = 40$$

In [8]:
(F.grade(4) / F.grade(2)).dual().norm()

40.0

In order to compute the deviation in this distance due to tolerances, we take $f_0$ as our reference and compute the effect $T$ has on $f_5$ at first order as
$$ f_5'= f_5 + T \times f_5 $$
and compute the modified FR as
$$ F' = f_5' \widetilde{f_0} = F + (T \times f_5) \widetilde{f_0} \ . $$

In [9]:
Fp = (f5 + T.cp(f5)) * ~f0
Fp

(55.0*a1) 𝐞₀₁ + (55.0*b1 - 85.0*c2 - 85.0*c3 + u2 + u3 + u4) 𝐞₀₂ + 1 𝐞₁₂ + (110.0*a1 + 85.0*a2 + 85.0*a3 + w1 + w2 + w3 + w4 - 40) 𝐞₀₁₂₃

We can now recompute the new distance from the modified FR $F'$: 

In [10]:
F2 = Fp.grade(2)
F4 = Fp.grade(4)
(F4/F2).dual().norm()

((12100.0*a1**2 + 18700.0*a1*a2 + 18700.0*a1*a3 + 220.0*a1*w1 + 220.0*a1*w2 + 220.0*a1*w3 + 220.0*a1*w4 - 8800.0*a1 + 7225.0*a2**2 + 14450.0*a2*a3 + 170.0*a2*w1 + 170.0*a2*w2 + 170.0*a2*w3 + 170.0*a2*w4 - 6800.0*a2 + 7225.0*a3**2 + 170.0*a3*w1 + 170.0*a3*w2 + 170.0*a3*w3 + 170.0*a3*w4 - 6800.0*a3 + w1**2 + 2*w1*w2 + 2*w1*w3 + 2*w1*w4 - 80*w1 + w2**2 + 2*w2*w3 + 2*w2*w4 - 80*w2 + w3**2 + 2*w3*w4 - 80*w3 + w4**2 - 80*w4 + 1600)**0.5)

It is a bit hard to make sence of this answer however, because it seems sympy is unable to simplify multivariate polynomials, but if we go through the steps one at a time we will see that the true answer is not actually that complicated. Looking at $\langle F \rangle_4 / \langle F \rangle_2$, we see that the $\mathbf{e}_{12}$ coefficient is identical to the $\mathbf{e}_{03}$ coefficient of $T$ minus $40$:

In [11]:
(F4/F2).dual(), T.e03 - 40

((110.0*a1 + 85.0*a2 + 85.0*a3 + w1 + w2 + w3 + w4 - 40) 𝐞₁₂,
 110.0*a1 + 85.0*a2 + 85.0*a3 + w1 + w2 + w3 + w4 - 40)

So the complicated mess above is simply the result of squaring this expression and taking its square root, which is evidently
$$  d = \lvert w_1 + w_2 + w_3 + w_4 + 110 \alpha_1 + 85\alpha_2 + 85\alpha_3 - 40 \rvert \ .$$
This is the expected result, because the FR is aligned with the $z$-axis and so the new distance under the influence of the tolerance zones captured by $T$, is to shift the original distance of $40$ by an amount $w_1 + w_2 + w_3 + w_4 + 110 \alpha_1 + 85\alpha_2 + 85\alpha_3$.
